# Encoding 방식 정리

인코딩 방법중 가장 basic 한 방법은 one-hot encoding과 label encoding이 있다. 그 둘에 대한 이야기는 생략하고 Binary encoding과 BaseN encoding, Hashmap encoding 에 대해 공부해보고자 한다.  


## Cardinality
우선 알아야 할 개념은 cardinality 이다.   
결론부터 말하자면, *중복도가 '낮으면' cardinality가 '높다' 고 하고, 중복도가 '높으면' cardinality가 '낮다'*고 한다. 
> 즉, 카디널리티는 전체 행에 대한 특정 컬럼의 중복 수치를 나타내는 지표이다.


카디널리티는 객관적 수치보다는 상대적인 개념으로 이해해야 한다.   
인덱스를 걸 때, 내가 원하는 데이터를 선택하는 과정에서 최대한 많은 데이터가 걸려져야 성능이 좋을 것이다. 즉, 여러 걸럼을 동시에 인덱싱할 때, 다음과 같이 카디널리티가 높은 컬럼(중복이 적은 컬럼)을 우선순위를 두는것이 인덱싱 전략에 유리하다.

# 요약

## Classic Encoder

하나의 열(순서형)에서 최대 k개의 열(OneHot)에 정보를 포함하는 연속체에서 볼 수 있다. 가장 기본형이다. 

 * Ordinal - 문자열 레이블을 1에서 k 까지의 정수값으로 변환한다. 순서형에서 자주쓰인다.
 * OneHot - 다른 모든 값과 비교할 각 값에 대해 하나의 열이다. 명목형에서 주로 쓰이지만 순서형에도 쓰인다.
 * Binary - 각 정수를 2진 숫자로 변환한다. 각 이진수는 하나의 열을 갖고 정보 손실이 있지만 차원이 적다. 순서형에서 쓰인다.
 * BaseN - 순서형, 이진법 또는 더 많은 차원의 인코딩을 한다. 많은 기능을 갖고있진 않으며 잘 쓰이지않는다.
 * Hasing - onehot과 비스샇지만 크기가 적고 충돌로 인해 일부 정보가 손실된다.
 * Sum - oneHot과 비슷하지만 하나의 값이 일정하게 유지되고, 모든 열에서 -1로 인코딩 된다.

## Contrast Encoder

 * Helmert
 * Backward Differnce
 * Polynomial

## Bayesian Encoder
베이지안 인코더는 인코딩에 종속 변수의 정보를 사용한다. 하나의 열을 출력하고 높은 카디널리티 데이터와 잘 작동 할 수 있다.

 * Target - DV 의 평균을 사용하고 과적합/응답 누출을 방지하기위한 조치를 취해야한다. 명목, 순서형, 분류 작업용.
 * LeaveOneOut - Target과 유사하지만 손실을 방지한다. 명목형, 서수형, 분류형.
 * WeightOfEvidence - https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html


# category_encoders
sklearn의 Categorical Endoding Methods 중 하나. 이는 범주형 자료에 대한 인코딩을 전문적으로 다루기때문에 sklearn의 기본 패키지 (sklearn.preprocessing)보다 뛰어난 장점이 많다.   
category_encoder은 DataFrame으로 결과를 반환 할 분 아니라, 컬럼에 각각 원래의 변수이름이 표현된다. 

input으로 받는 type은 다음과 같이 구별된다. 
  1. sklearn.preprocessing : DataFrame, (-1,1)꼴의 array
  2. category_encoders: DataFrame, Series



In [ ]:
pip install category_encoders
import category_encoders as ce


## Binary Encoder 
2진법 인코더는 각자릿수마다 0과 1 두가지 숫자로 표현한다. 100가지의 피쳐를 가지는 범주형 변수를 Onehot encoding 하면 100차원의 벡터 혹은 100번의 획순이 필요하다. 하지만 binary 는 단 7자리로 표현이 가능하기때문에 획기적인 차원축소를 할 수 있다. 

구체적인 과정은 아래와 같다.
1. numeric value로 바꿔주기
2. 이진법 숫자로 바꿔주기
3. 각 자릿수에 맞춰 컬럼을 만들어주기

> 1번 과정을 할 때에는 *Ordinal Encoder* 를 사용한다. 여러 이유가 있겠지만 Label 인코딩이 0부터 시작하는 반면 Ordinal 인코딩은 1부터 시작하기때문. 


In [ ]:
# Binary Encoder 불러오기
from category_encoders import *

UserName = BinaryEncoder(cols = ['RAD'], drop_invariant=True)
UserName

자신이 원하는 옵션을 결정하는 곳. 위 처럼만 코딩을 해주어도 새로운 변수를 0 벡터꼴로 바꿔주기 때문에 차원변화없이 문제를 잘 해결 할 수 있습니다. (drop_invariant는 모두 하나의 값으로만 이루어진 컬럼을 제거하는 기능 수행.) 

In [ ]:
UserName.fit_transform(UserData)


 * fit_transform : fit 과 transform의 기능 합친것. 인코딩을 원하는 데이터를 집어넣는 첫 단계이고, 결과가 위에서 생성한 instance에 저장된다.(UserName)

 * transform : 앞서 인코딩한 방식에 새로운 데이터셋을 집어넣어 같은 방식으로 변환한다.
 * inverse_transform : 원래 데이터가 궁금할때.

# BaseN

앞의 Binary는 BaseN 인코딩에서 N을 2로 설정한 것과 동일.

----
1. N=1: OneHot 인코딩 (정확히 같진 않지만 사실상같다고 봐도 무방.)
2. N=2 : Binary 인코딩
3. ...
4. N=N : Ordinary 인코딩 

----

보통은 N차원까지 가지않는다 그 이유는 아래와 같다.
1. 3진법 이상부터는 0과 1이 아닌 더 큰 숫자가 등장한다. 이렇게 되면 하나의 컬럼에 대소관계가 드러나게되기때문에 순서형데이터처럼 취급 될 가능성이 있다.
2. 2진법으로도 충분히 차원을 축소할 수 있기 때문이다. 

# Hasing

원핫 인코딩과 유사하지만, 새로운 차원이 더 적고 충돌로 인해 정보가 손실된다. 겹치는 부분이 많지않으면 충돌이 성능에 큰 영향을 주지 않는다. 

In [ ]:
ce_hash=ce.HashingEncoder(cols=['color'])
ce_hash.fit_transform(X,y)

n_components 확장 열 수를 제어하는 매개변수이다. 기본값은 8개 열이고, 3개의 값이 있는 예제열에서는 5개 칸을 전부 쓰는 열이 된다. 기본값은md5. 


> **명목형 변수나 순서형 변수가 high cardinality feature인 경우, hashing Encoder을 사용해볼 만 하다.**

# 결론

1. **명목형(순위가 없는) 변수일 경우** OneHot, Hasing , LeaveOneOut, Target 인코딩을 사용하되, 중복이 적은(카디널리티가 높은 열에는) OneHot을 사용하지 않는것이 좋다. 차원이 엄청나게 늘어나기때문이다.

2. **순서형(순위가 있는) 변수일 경우** Ordinary,Binary,Onehot,LeaveOneOut 및 Target을 사용하자.